In [1]:
import numpy as np
import pandas as pd
import datetime
from collections import defaultdict
import pybgpstream

In [6]:
stream = pybgpstream.BGPStream(
    # Consider this time interval:
    # Sat, 01 Aug 2015 7:50:00 GMT -  08:10:00 GMT
    # from_time="2015-08-01 07:50:00", until_time="2015-08-01 08:10:00",
    from_time="2014-04-01 16:00:00", until_time="2014-04-01 17:00:00",
    collectors=["rrc06"],
    record_type="updates",
    filter="ipversion 4"
)

# r = next(stream.records())


In [7]:

# <prefix, origin-ASns-set > dictionary
#prefix_origin = defaultdict(set)
# records = []
raw = []
# for i in [1,2,3]:
#     for rec in stream.records():
#         #print(rec.time, rec.dump_time)
#         print(rec.time)
for rec in stream.records():
    for elem in rec:
        # Get the prefix
        # pfx = elem.fields["prefix"]
        # print(pfx)
        
        if elem.type == "A":
            #print(elem.type, elem.time, elem.peer_asn, elem.peer_address, elem.fields["prefix"], elem.fields["next-hop"], elem.fields["as-path"])
            #records.append([elem.type, datetime.datetime.utcfromtimestamp(elem.time), np.uint16(elem.peer_asn), elem.peer_address, elem.fields["prefix"], np.array(elem.fields["as-path"].replace('{','').replace('}','').split(' '), dtype=np.uint16)])
            raw.append([elem.type, elem.time, elem.peer_asn, elem.peer_address, elem.fields["prefix"], elem.fields["as-path"]])
            #records.append([elem.type, elem.time, elem.peer_asn, elem.fields["prefix"], elem.fields["next-hop"]])
        elif elem.type == "W":
            raw.append([elem.type, elem.time, elem.peer_asn, elem.peer_address, elem.fields["prefix"], np.array([])])
             
#update|A|1438415697.000000|ris|rrc00|None|None|3549|208.51.134.248|194.33.126.0/24|208.51.134.248|3549 3356 29049 12880 42337 47843 58249|3549:2177 3549:30840|None|None

In [8]:
dfr = pd.DataFrame(data=raw, columns=["type", "time", "peer_asn", "peer_ip", "prefix", "as_path"])
dfr.info()
dfr

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20709 entries, 0 to 20708
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   type      20709 non-null  object 
 1   time      20709 non-null  float64
 2   peer_asn  20709 non-null  int64  
 3   peer_ip   20709 non-null  object 
 4   prefix    20709 non-null  object 
 5   as_path   20709 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 970.9+ KB


,type,time,peer_asn,peer_ip,prefix,as_path
0,A,1.396368e+09,25152,202.249.2.185,177.107.11.0/24,25152 2914 1239 16735 53059 262730
1,A,1.396368e+09,25152,202.249.2.185,190.211.244.0/24,25152 6939 12956 22927 52295
2,A,1.396368e+09,25152,202.249.2.185,190.211.245.0/24,25152 6939 12956 22927 52295
3,A,1.396368e+09,25152,202.249.2.185,177.107.11.0/24,25152 6939 16735 53059 262730
4,A,1.396368e+09,25152,202.249.2.185,177.107.11.0/24,25152 6939 3549 16735 53059 262730
...,...,...,...,...,...,...
20704,A,1.396372e+09,25152,202.249.2.185,94.139.170.0/23,25152 6939 8529 12880 25124 57218 57218 57218
20705,A,1.396372e+09,25152,202.249.2.185,188.253.120.0/22,25152 6939 8529 48159 58127
20706,A,1.396372e+09,25152,202.249.2.185,5.233.125.0/24,25152 6939 8529 48159
20707,A,1.396372e+09,25152,202.249.2.185,66.210.60.0/24,25152 6939 22773 20450


In [9]:
dfr.to_hdf("../data/01_raw/googletest-small.h5", key='raw')

/tmp/ipykernel_4097/295202921.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['type', 'peer_ip', 'prefix', 'as_path'], dtype='object')]

  dfr.to_hdf("googletest-small.h5", key='raw')
